In [2]:
! python --version

Python 3.8.19


In [1]:
# from espnet2.bin.tts_inference import Text2Speech
# from espnet2.utils.types import str_or_none
from IPython.display import Audio
from urllib.parse import quote
from pydub import AudioSegment
from bs4 import BeautifulSoup
from tqdm import tqdm
from whisper.audio import load_audio
import io
import time
import json
import torch
import requests
import warnings
import soundfile as sf
import numpy as np
from numpy.linalg import norm
import base64
import requests
import simpleaudio as sa
from scipy.signal import resample

warnings.simplefilter("ignore", UserWarning)

2024-05-24 19:44:47,860, INFO, logger.py, >2024-05-24_19-44-47.log


In [2]:
def get_tukuyomi(text:str, is_save=False):
    """
    ! python main.py
    で実行したAPIサーバーにリクエストを投げる
    """
    res = requests.get(f"http://localhost:5001/tts?text={quote(text)}")
    # 既に取得した res.content を使用
    if res.status_code == 200:
        audio = Audio(res.content, autoplay=True)
        display(audio)
        if is_save:
            with open(f"./data/{text.replace('/','_')}.wav", "wb") as f:
                f.write(res.content)
                print("ファイルを保存しました。")
        return res
    else:
        print(f"エラーが発生しました: ステータスコード {res.status_code}")
        
def get_vector(text:str):
    """
    言語のベクトル化
    """
    res = requests.get(f"http://localhost:5001/vector?text={quote(text)}")
    if res.status_code==200:
        vector_str = json.loads(json.loads(res.text))
        vector = np.array(vector_str)
        return vector
    else:
        return res.error

def cos_similar(A,B):
    return np.dot(A,B)/(norm(A)*norm(B))

def resample_audio(audio_data, original_sr, target_sr):
    # サンプル数を計算
    num_samples = int(len(audio_data) * float(target_sr) / original_sr)
    # リサンプリング
    resampled_data = resample(audio_data, num_samples)
    return resampled_data

def get_transcribe(file_path, target_sr=16000):
    # WAVファイルを読み込む
    wave_obj = sa.WaveObject.from_wave_file(file_path)
    play_obj = wav_obj.play()
    
    # オーディオデータをNumPy配列に変換し、int16型にキャスト
    audio_data = np.frombuffer(wave_obj.audio_data, dtype=np.int16)
    
    # サンプリングレートを変更
    resampled_audio_data = resample_audio(audio_data, wave_obj.sample_rate, target_sr)
    
    # NumPy配列をバイトデータに変換
    audio_bytes = resampled_audio_data.astype(np.int16).tobytes()
    
    # バイトデータをBase64にエンコード
    audio_base64 = base64.b64encode(audio_bytes).decode('utf-8')

    # APIのURL
    url = 'http://localhost:5001/transcribe'
    
    # POSTリクエストの送信
    response = requests.post(url, json={'audio': f"data:audio/wav;base64,{audio_base64}"})
    if response.status_code==200:
        return response.json()["text"]
    else:
        return response.__dict__

def get_morphologic(text:str):
    """
    文章の形態素解析
    """
    res = requests.get(f"http://localhost:5001/morphologic?text={quote(text)}")
    if res.status_code==200:
        vector_str = json.loads(json.loads(res.text))
        vector = np.array(vector_str)
        return vector
    else:
        return res.error

In [10]:
res = get_tukuyomi("何の音使います？")
# url = 'http://localhost:3030/speak/'
# files = {'audioFile': ('audio.wav', res.content)}

# response = requests.post(url, files=files)
# print(response.text)


In [3]:
response = requests.post("http://localhost:5001/tts/", data={"text":"海は嫌いです。潮風がベタついてたまりません。"})
print(response.text)

ConnectionError: HTTPConnectionPool(host='localhost', port=3030): Max retries exceeded with url: /tts/ (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x1036810d0>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [ ]:
quotes = []
for i in tqdm(range(1,18)):
    res = requests.get(f"https://iyashitour.com/meigen/theme/life/{i}")
    soup = BeautifulSoup(res.text, 'html.parser')
    quote_elms = soup.find_all('p', {'class':'quote'})
    quotes += [q.get_text(" ", strip=True) for q in quote_elms]

with open('./data/quotes.txt', 'w') as f:
    for line in quotes:
        f.write(f"{line}\n")

In [68]:
with open("./data/55_富者の秘訣.txt","r") as f:
    # 最初の音声ファイルを基点として、他の音声データを結合します
    quotes = [q for q in f]
    combined = AudioSegment.empty()
    a = AudioSegment.silent(duration=1000)
    for text in tqdm(quotes):
        if text=="\n":
            combined += a
            continue
        res = requests.get(f"http://localhost:5001/tts?text={quote(text)}")
        if res.status_code == 200:
            content = res.content
            audio_segment = AudioSegment.from_file(io.BytesIO(content), format="wav")
            # 無声音
            combined += audio_segment+a
    # 結合した音声ファイルを新しいファイルとして出力
    combined.export("./data/55_富者の秘訣.wav", format="wav")

100%|██████████| 363/363 [06:06<00:00,  1.01s/it]


In [67]:
with open("./data/55_富者の秘訣.txt","r") as f:
    # 最初の音声ファイルを基点として、他の音声データを結合します
    quotes = [q for q in f]
    combined = AudioSegment.empty()
    for text in tqdm(quotes):
        if text=="\n":
            break

  1%|          | 3/363 [00:00<00:00, 61082.10it/s]


In [66]:
quotes[3]

'\n'

In [ ]:
Audio("./data/quotes.wav")

In [58]:
vec = get_vector("豆")
vec.shape

(768,)

In [43]:
vec_1 = get_vector("カラス")
vec_1.shape

(768,)

In [44]:
cos_similar(vec, vec_1)

0.7976341244519816

In [46]:
# 関数を呼び出し
k = get_transcribe(file_path)
k

'海は嫌いです。潮風がベタついてたまりません。'

In [47]:
res = requests.get(f"http://localhost:5001/tokenize?text={quote(k)}")
res.json()

'{"input_ids": [[2, 295, 9, 12844, 2992, 8, 6170, 29122, 14, 298, 28502, 7071, 16, 6918, 28477, 6769, 1058, 8, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], "token_type_ids": [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [3]:
from logger import logger

In [6]:
with open("./data/quotes.txt") as f:
    for l in f:
        vec = get_vector(l)
        li = l.replace('\n','')
        logger.info(f"{str(vec.shape).replace(',','_')}, {', '.join(str(x) for x in vec.tolist())}, {li}")

In [4]:
get_morphologic("海は嫌いです。潮風がベタついてたまりません。")

array(['海', 'は', '嫌い', 'です', '。', '潮風', 'が', 'ベタつい', 'て', 'たまり', 'ませ',
       'ん', '。'], dtype='<U4')

In [48]:
from datasets import load_dataset

dataset = load_dataset("yuzuai/rakuda-questions")

print(dataset)
# => DatasetDict({
#       train: Dataset({
#           features: ['category', 'question_id', 'text'],
#           num_rows: 40
#       })
#    })


/Users/matsuzawakazuki/miniforge3/envs/.tts/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 100%|██████████| 40/40 [00:00<00:00, 3464.79 examples/s]

DatasetDict({
    train: Dataset({
        features: ['category', 'question_id', 'text'],
        num_rows: 40
    })
})


In [57]:
dataset = load_dataset("izumi-lab/llm-japanese-dataset", revision="main")

Generating train split: 9074340 examples [00:03, 2344771.84 examples/s]


In [60]:
df_train = pd.DataFrame(dataset["train"])
df_train.head()

,instruction,input,output
0,「abc ～the first～」へようこそ！さて、ABC・・・と始まるアルファベットは、全...,,26文字
1,人気漫画『ドラえもん』の登場人物で、ジャイアンの苗字は剛田ですが、スネ夫の苗字は何でしょう？,,骨川（滑川も正解）
2,格闘家ボブ・サップの出身国はどこでしょう？,,アメリカ
3,ロシア語で「城」という意味がある、ロシアの大統領府の別名は何でしょう？,,クレムリン
4,織田信長、豊臣秀吉、徳川家康という３人の戦国武将の性格を表現するのに用いられる鳥は何でしょう？,,ホトトギス


In [61]:
df_train.to_csv("./data/train.csv")